# **Train Artificial Neural Networks**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import pickle 
from tensorflow.keras.layers import BatchNormalization


#INPUTS of ANN :  Water (lb/ft3)--Cement (lb/ft3)--Slag (lb/ft3)--Fly Ash (lb/ft3)--Silica Fume (lb/ft3)--Fine Aggregate (lb/ft3)--Coarse Aggregate (lb/ft3)--Superplasticizer (lb/ft3)--Fresh Density (lb/ft3)--Comp. Str. test age (days)
#Outputs of ANN : Compressive Strength (Psi)--CO2 Emission (lb/ft3)

data = pd.read_excel('/kaggle/input/dataset/Main_Dataset.xlsx')

X = data.drop(['Compressive Strength (Psi)', 'CO2 Emission (lb/ft3)'], axis=1)
y = data[['Compressive Strength (Psi)', 'CO2 Emission (lb/ft3)']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_scaler_comp_strength = StandardScaler()
y_train_scaled_comp_strength = y_scaler_comp_strength.fit_transform(y_train[['Compressive Strength (Psi)']])
y_test_scaled_comp_strength = y_scaler_comp_strength.transform(y_test[['Compressive Strength (Psi)']])

y_scaler_co2 = StandardScaler()
y_train_scaled_co2 = y_scaler_co2.fit_transform(y_train[['CO2 Emission (lb/ft3)']])
y_test_scaled_co2 = y_scaler_co2.transform(y_test[['CO2 Emission (lb/ft3)']])

y_train_scaled = np.concatenate((y_train_scaled_comp_strength, y_train_scaled_co2), axis=1)
y_test_scaled = np.concatenate((y_test_scaled_comp_strength, y_test_scaled_co2), axis=1)

def model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_shape,), kernel_regularizer=l2(0.01)),
        BatchNormalization(), 
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        tf.keras.layers.Dense(2)  
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse',metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

model = model(X_train_scaled.shape[1])

early_stopping = EarlyStopping(monitor='val_loss', patience=80, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, 
                    epochs=1500, 
                    batch_size=64, 
                    validation_data=(X_test_scaled, y_test_scaled), 
                    callbacks=[early_stopping],
                    verbose=1)

model.save('concrete_prediction_model.keras')

with open('x_scaler.pkl', 'wb') as f:
    pickle.dump(x_scaler, f)

with open('y_scaler_comp_strength.pkl', 'wb') as f:
    pickle.dump(y_scaler_comp_strength, f)

with open('y_scaler_co2.pkl', 'wb') as f:
    pickle.dump(y_scaler_co2, f)

# **Model Evaluation**

In [ ]:
from sklearn.metrics import r2_score 
y_pred_scaled = model.predict(X_test_scaled)

y_pred_comp_strength = y_scaler_comp_strength.inverse_transform(y_pred_scaled[:, 0].reshape(-1, 1))
y_true_comp_strength = y_scaler_comp_strength.inverse_transform(y_test_scaled_comp_strength)

y_pred_co2 = y_scaler_co2.inverse_transform(y_pred_scaled[:, 1].reshape(-1, 1))
y_true_co2 = y_scaler_co2.inverse_transform(y_test_scaled_co2)

r2_comp_strength = r2_score(y_true_comp_strength, y_pred_comp_strength)
r2_co2 = r2_score(y_true_co2, y_pred_co2)

print(f"Model Evaluation")
loss, mse, mae, rmse = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"R2 Score for Compressive Strength: {r2_comp_strength:.4f}")
print(f"R2 Score for CO2 Emission: {r2_co2:.4f}")

# **Plot Metrics**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

output_dir = 'training_figures'
os.makedirs(output_dir, exist_ok=True)

def plot_metrics():
    
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
    
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.grid(False, alpha=0.3)
    
    
    loss_filename = os.path.join(output_dir, 'training_validation_loss.png')
    plt.savefig(loss_filename, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Loss figure saved as: {loss_filename}")
    plt.show()
    
    
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['mae'], label='Training MAE', color='green', linewidth=2)
    plt.plot(history.history['val_mae'], label='Validation MAE', color='orange', linewidth=2)

    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Mean Absolute Error', fontsize=14)
    plt.legend(fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.grid(False, alpha=0.3)
    
    mae_filename = os.path.join(output_dir, 'training_validation_mae.png')
    plt.savefig(mae_filename, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"MAE figure saved as: {mae_filename}")
    plt.show()
    
    
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['root_mean_squared_error'], label='Training RMSE', color='purple', linewidth=2)
    plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE', color='brown', linewidth=2)

    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Root Mean Squared Error', fontsize=14)
    plt.legend(fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.grid(False, alpha=0.3)
    
    rmse_filename = os.path.join(output_dir, 'training_validation_rmse.png')
    plt.savefig(rmse_filename, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"RMSE figure saved as: {rmse_filename}")
    plt.show()
    
    
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['mse'], label='Training MSE', color='teal', linewidth=2)
    plt.plot(history.history['val_mse'], label='Validation MSE', color='magenta', linewidth=2)

    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Mean Squared Error', fontsize=14)
    plt.legend(fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.grid(False, alpha=0.3)
    
    mse_filename = os.path.join(output_dir, 'training_validation_mse.png')
    plt.savefig(mse_filename, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"MSE figure saved as: {mse_filename}")
    plt.show()

plot_metrics()